# coursera NN and DL
### Week 2



In [1]:
import tensorflow as tf
import numpy as np

In [20]:
from tensorflow.keras import layers

In [125]:
a = np.array([1,2,3,4])
print(a)

[1 2 3 4]


In [131]:
import time

a = np.random.rand(1000000)
b = np.random.rand(1000000)

tic = time.time()
c = np.dot(a,b)
toc = time.time()
print(str(1000*(toc-tic)))

c = 0
tic = time.time()
for i in range(1000000):
    c += a[i]*b[i]
toc = time.time()
print(str(1000*(toc-tic)))

9.022235870361328
1108.9260578155518


주피터 환경에서는 cpu가 주로 사용됨. gpu나 cpu 모두 병렬 연산이 수행되는데, (SIMD)
이는 numpy나 python으로 하여금 한 지시로 병렬연산을 하여 훨씬 빠르게 처리할 수 있음을 나타냄

In [132]:
a = np.random.randn(3, 3)
b = np.random.randn(3, 1)
c = a*b

formula is: $$sigmoid\_derivative(x) = \sigma'(x) = \sigma(x) (1 - \sigma(x))\tag{2}$$

Week 2까지의 내용을 모두 해결하였음
(assignment 1, 2를 제출, 클리어)
<br>
근데..
<br>
이게 과연 도움이 될지는 모름.
<br>
그래도 일단 오늘 이해한 것을 적어보는 시간을 가지겠음.

### logistic regression 에서의 loss 및 cost로 학습하기

In [137]:
import os
import pickle

In [139]:
x = open('/root/storage/DATA/pickle/checked','rb')
data = pickle.load(x)
x.close()

In [140]:
data.shape

(25, 200, 80, 97)

In [146]:
cycle = np.random.permutation(25)
data_1 = cycle[:5]
data_2 = cycle[5:10]

true_data = np.ndarray((5,200,80,97))
false_data = np.ndarray((5,200,80,97))
for i in range(5):
    true_data[i] = data[data_1[i]]
    false_data[i] = data[data_2[i]]

In [270]:
true_X = true_data.reshape(true_data.shape[0]*true_data.shape[1],
                          true_data.shape[2]*true_data.shape[3])
false_X = false_data.reshape(false_data.shape[0]*false_data.shape[1],
                          false_data.shape[2]*false_data.shape[3])
true_Y = np.ones((true_X.shape[0],1))
false_Y = np.zeros((false_X.shape[0],1))
print(true_X.shape,false_X.shape,true_Y.shape,false_Y.shape)
all_x = np.concatenate((true_X,false_X))
all_y = np.concatenate((true_Y,false_Y))

#all_x = np.random.shuffle(all_x[:])
#all_y = np.random.shuffle(all_y[:])

from sklearn.model_selection import train_test_split

train_x , test_x , train_y, test_y = train_test_split(all_x,all_y,test_size=0.1)
print(train_x.shape,test_x.shape,train_y.shape,test_y.shape)
train_x = train_x.reshape(train_x.shape[1],train_x.shape[0])
test_x = test_x.reshape(test_x.shape[1],test_x.shape[0])
train_y = train_y.reshape(train_y.shape[1],train_y.shape[0])
test_y = test_y.reshape(test_y.shape[1],test_y.shape[0])
print(train_x.shape,test_x.shape,train_y.shape,test_y.shape)

(1000, 7760) (1000, 7760) (1000, 1) (1000, 1)
(1800, 7760) (200, 7760) (1800, 1) (200, 1)
(7760, 1800) (7760, 200) (1, 1800) (1, 200)


In [316]:
learning_rate = 0.01
iteration = 2000
w = np.zeros((train_x.shape[0],1))
b = 0

In [317]:
def sigmoid(x):
    x = 1/(1+np.exp(-x))
    return x

In [318]:
m = train_x.shape[1]
for i in range(iteration):
    A = sigmoid(np.dot(w.T,train_x)+b)
    cost = -np.sum(np.dot(train_y,np.log(A.T))+np.dot((1-train_y),np.log(1-A.T)))/m
    
    dw = np.dot(train_x,(A-train_y).T)/m
    db = np.sum((A-train_y))/m
    
    cost = np.squeeze(cost)
    
    w = w - learning_rate * dw
    b = b - learning_rate * db
    
    if i%100==0:
        print(cost)
        m2 = test_x.shape[1]
        prediction = np.zeros((1,m2))
        
        w2 = w.reshape(test_x.shape[0],1)
        A2 = sigmoid(np.dot(w2.T,test_x)+b)
        for j in range(A2.shape[1]):
            if A2[0,j]>=0.5:
                prediction[0,j] = 1
            else:
                prediction[0,j] = 0
        print('test accuracy : {}%'.format(100-np.mean(np.abs(prediction-test_y))*100))
        m2 = train_x.shape[1]
        prediction = np.zeros((1,m2))
        
        w2 = w.reshape(train_x.shape[0],1)
        A2 = sigmoid(np.dot(w2.T,train_x)+b)
        for j in range(A2.shape[1]):
            if A2[0,j]>=0.5:
                prediction[0,j] = 1
            else:
                prediction[0,j] = 0
        print('train accuracy : {}%'.format(100-np.mean(np.abs(prediction-train_y))*100))

0.6931471805599446
test accuracy : 53.5%
train accuracy : 54.55555555555556%
0.5768016817060015
test accuracy : 53.5%
train accuracy : 73.16666666666667%
0.5248149221667037
test accuracy : 51.5%
train accuracy : 78.44444444444444%
0.4863994721580847
test accuracy : 51.5%
train accuracy : 82.38888888888889%
0.4556013483976805
test accuracy : 51.5%
train accuracy : 84.55555555555556%
0.42984824000803146
test accuracy : 51.5%
train accuracy : 86.61111111111111%
0.40772524109777186
test accuracy : 53.0%
train accuracy : 88.0%
0.3883541687025399
test accuracy : 54.5%
train accuracy : 89.16666666666667%
0.3711473118227767
test accuracy : 53.5%
train accuracy : 89.77777777777777%
0.3556901681468778
test accuracy : 52.0%
train accuracy : 90.83333333333333%
0.3416787043482114
test accuracy : 51.5%
train accuracy : 91.66666666666667%
0.3288829412398794
test accuracy : 50.5%
train accuracy : 92.5%
0.3171245039783114
test accuracy : 51.0%
train accuracy : 93.22222222222223%
0.3062621201952068
test

### 결과

일단 뭐 원리에 대해서는 알 수 있었던 시간이었다.<br>
logistic regression이었기 때문에 내가 주었던 옵션, 즉 5개의 클래스는 참, 5개의 클래스는 거짓이<br>
test 단계에서 학습이 전혀 이루어 지지 않았음을 알 수 있다. 이는,<br>
짐작하건데 클래스 개수를 두개씩으로 줄이면 가능하지 않을까 생각한다. 내일은 두개로 하는것을 간단히 해보고<br>
다음 단계로 넘어갈 것이당